# Análisis de Sensibilidad Paramétrica con Widgets Interactivos

Este notebook demuestra cómo usar **ipywidgets** para crear análisis de sensibilidad paramétrica interactiva en PetroKit.

**Fase 3 - Profesionalización**: Ejemplo avanzado con widgets interactivos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown
from petrokit.ipr import ipr_curve_vogel, ipr_curve_fetkovich
from petrokit.vlp import vlp_curve
from petrokit.nodal import nodal_analysis

# Configuración de estilo
plt.style.use('seaborn-darkgrid')

## 1. Sensibilidad de IPR (Modelo Vogel)

Analiza cómo varía la curva IPR al cambiar el caudal máximo (q_max).

In [ ]:
def plot_ipr_sensitivity(q_max, p_res):
    """Graficar IPR con parámetros ajustables"""
    pwf, q = ipr_curve_vogel(p_res, q_max)
    
    plt.figure(figsize=(10, 6))
    plt.plot(q, pwf, 'b-', linewidth=2.5, label='IPR - Vogel')
    plt.xlabel('Caudal (STB/d)', fontsize=12)
    plt.ylabel('Presión de fondo (psi)', fontsize=12)
    plt.title(f'Curva IPR - Sensibilidad\nq_max = {q_max} STB/d, p_res = {p_res} psi', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=11)
    plt.xlim(0, q_max * 1.1)
    plt.ylim(0, p_res * 1.1)
    plt.tight_layout()
    plt.show()

# Crear widget interactivo
interact(plot_ipr_sensitivity,
         q_max=FloatSlider(min=500, max=2500, step=100, value=1200, description='q_max (STB/d):'),
         p_res=FloatSlider(min=1500, max=4500, step=100, value=3000, description='p_res (psi):'));

## 2. Sensibilidad de VLP (Diámetro de Tubing)

Analiza el efecto del diámetro del tubing en la curva VLP.

In [ ]:
def plot_vlp_sensitivity(diameter, well_depth, rho):
    """Graficar VLP con diferentes diámetros"""
    q_range = np.linspace(0, 1500, 50)
    pwf = vlp_curve(q_range, well_depth, rho, mu=1.0, d=diameter)
    
    plt.figure(figsize=(10, 6))
    plt.plot(q_range, pwf, 'r-', linewidth=2.5, label=f'VLP (d={diameter} in)')
    plt.xlabel('Caudal (STB/d)', fontsize=12)
    plt.ylabel('Presión de fondo (psi)', fontsize=12)
    plt.title(f'Curva VLP - Sensibilidad al Diámetro\nProfundidad = {well_depth} ft, ρ = {rho} lb/ft³', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=11)
    plt.xlim(0, 1500)
    plt.tight_layout()
    plt.show()

# Widget interactivo
interact(plot_vlp_sensitivity,
         diameter=FloatSlider(min=1.5, max=4.5, step=0.25, value=2.992, description='Diámetro (in):'),
         well_depth=FloatSlider(min=4000, max=12000, step=500, value=8000, description='Profundidad (ft):'),
         rho=FloatSlider(min=40, max=80, step=5, value=60, description='Densidad (lb/ft³):'));

## 3. Análisis Nodal Completo - Sensibilidad Integrada

Combina IPR y VLP para analizar el punto de operación con diferentes parámetros.

In [ ]:
def plot_nodal_sensitivity(q_max, diameter, well_depth):
    """Análisis nodal con parámetros ajustables"""
    p_res = 3000
    rho = 60
    mu = 1.0
    
    # Calcular curvas
    pwf_ipr, q_ipr = ipr_curve_vogel(p_res, q_max)
    q_vlp = np.linspace(0, q_max, 50)
    pwf_vlp = vlp_curve(q_vlp, well_depth, rho, mu, diameter)
    
    # Calcular punto de operación
    q_op, pwf_op = nodal_analysis(p_res, q_max, well_depth, rho, mu, diameter)
    
    # Graficar
    plt.figure(figsize=(12, 7))
    plt.plot(q_ipr, pwf_ipr, 'b-', linewidth=2.5, label='IPR - Vogel')
    plt.plot(q_vlp, pwf_vlp, 'r-', linewidth=2.5, label='VLP')
    
    if q_op > 0:
        plt.plot(q_op, pwf_op, 'go', markersize=12, label=f'Punto de operación\nQ = {q_op:.1f} STB/d\nPwf = {pwf_op:.1f} psi', zorder=5)
    
    plt.xlabel('Caudal (STB/d)', fontsize=12)
    plt.ylabel('Presión de fondo (psi)', fontsize=12)
    plt.title(f'Análisis Nodal - Sensibilidad Paramétrica\nq_max={q_max} STB/d, d={diameter} in, H={well_depth} ft', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=11, loc='best')
    plt.xlim(0, q_max * 1.1)
    plt.ylim(0, p_res * 1.1)
    plt.tight_layout()
    plt.show()
    
    # Mostrar resultados
    if q_op > 0:
        print(f"✓ Punto de operación encontrado:")
        print(f"  - Caudal de operación: {q_op:.2f} STB/d")
        print(f"  - Presión de fondo: {pwf_op:.2f} psi")
    else:
        print("⚠ No hay flujo: VLP(q=0) ≥ p_res")

# Widget interactivo completo
interact(plot_nodal_sensitivity,
         q_max=FloatSlider(min=600, max=2000, step=100, value=1200, description='q_max (STB/d):'),
         diameter=FloatSlider(min=2.0, max=4.0, step=0.25, value=2.992, description='Diámetro (in):'),
         well_depth=FloatSlider(min=5000, max=11000, step=500, value=8000, description='Profundidad (ft):'));

## 4. Comparación de Modelos IPR

Compara diferentes modelos IPR (Vogel vs Fetkovich) de manera interactiva.

In [ ]:
def compare_ipr_models(p_res, q_max_vogel, J_fetkovich):
    """Comparar modelos IPR"""
    # Vogel
    pwf_vogel, q_vogel = ipr_curve_vogel(p_res, q_max_vogel)
    
    # Fetkovich
    pwf_fetkovich, q_fetkovich = ipr_curve_fetkovich(p_res, J_fetkovich)
    
    # Graficar
    plt.figure(figsize=(12, 7))
    plt.plot(q_vogel, pwf_vogel, 'b-', linewidth=2.5, label=f'Vogel (q_max={q_max_vogel} STB/d)')
    plt.plot(q_fetkovich, pwf_fetkovich, 'g--', linewidth=2.5, label=f'Fetkovich (J={J_fetkovich} STB/d/psi)')
    
    plt.xlabel('Caudal (STB/d)', fontsize=12)
    plt.ylabel('Presión de fondo (psi)', fontsize=12)
    plt.title(f'Comparación de Modelos IPR\np_res = {p_res} psi', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=11)
    plt.xlim(0, max(q_vogel[-1], q_fetkovich[-1]) * 1.1)
    plt.ylim(0, p_res * 1.1)
    plt.tight_layout()
    plt.show()

# Widget interactivo
interact(compare_ipr_models,
         p_res=FloatSlider(min=2000, max=4000, step=100, value=3000, description='p_res (psi):'),
         q_max_vogel=FloatSlider(min=800, max=2000, step=100, value=1200, description='q_max Vogel:'),
         J_fetkovich=FloatSlider(min=0.2, max=2.0, step=0.1, value=0.5, description='J Fetkovich:'));

## Conclusiones

Este notebook demuestra cómo usar **ipywidgets** para:

1. ✅ Crear análisis de sensibilidad paramétrica interactivos
2. ✅ Visualizar el efecto de diferentes parámetros en tiempo real
3. ✅ Comparar diferentes modelos de manera interactiva
4. ✅ Facilitar el análisis de ingeniería sin necesidad de re-ejecutar código

**Fase 3 completada**: Ejemplos avanzados con ipywidgets ✓